## Amazon AgentCore Bedrock Code Interpreter를 사용한 고급 데이터 분석 - 튜토리얼(Langchain)
이 튜토리얼은 Python을 사용한 코드 실행을 통해 고급 데이터 분석을 수행하는 AI Agent를 생성하는 방법을 보여줍니다. LLM이 생성한 코드를 실행하기 위해 Amazon Bedrock AgentCore Code Interpreter를 사용합니다.

이 튜토리얼은 AgentCore Bedrock Code Interpreter를 사용하여 다음을 수행하는 방법을 보여줍니다:
1. 샌드박스 환경 설정
2. 사용자 쿼리를 기반으로 코드를 생성하여 고급 데이터 분석을 수행하는 langchain 기반 Agent 구성
3. Code Interpreter를 사용하여 샌드박스 환경에서 코드 실행
4. 결과를 사용자에게 다시 표시

## 사전 요구 사항
- Bedrock AgentCore Code Interpreter 액세스 권한이 있는 AWS 계정
- Code Interpreter 리소스를 생성하고 관리할 수 있는 필요한 IAM 권한 보유
- 필요한 Python 패키지 설치(boto3, bedrock-agentcore 및 langchain 포함)
- IAM 역할은 Amazon Bedrock에서 Model을 호출할 수 있는 권한이 있어야 함
 - US Oregon (us-west-2) 리전에서 Claude 3.7 Sonnet Model에 대한 액세스 권한

## IAM 실행 역할에 다음 IAM 정책이 연결되어 있어야 합니다

~~~ {
"Version": "2012-10-17",
"Statement": [
    {
        "Effect": "Allow",
        "Action": [
            "bedrock-agentcore:CreateCodeInterpreter",
            "bedrock-agentcore:StartCodeInterpreterSession",
            "bedrock-agentcore:InvokeCodeInterpreter",
            "bedrock-agentcore:StopCodeInterpreterSession",
            "bedrock-agentcore:DeleteCodeInterpreter",
            "bedrock-agentcore:ListCodeInterpreters",
            "bedrock-agentcore:GetCodeInterpreter"
        ],
        "Resource": "*"
    },
    {
        "Effect": "Allow",
        "Action": [
            "logs:CreateLogGroup",
            "logs:CreateLogStream",
            "logs:PutLogEvents"
        ],
        "Resource": "arn:aws:logs:*:*:log-group:/aws/bedrock-agentcore/code-interpreter*"
    }
]
}

## 작동 방식

코드 실행 샌드박스는 Code Interpreter, 셸 및 파일 시스템을 갖춘 격리된 환경을 생성하여 Agent가 사용자 쿼리를 안전하게 처리할 수 있도록 합니다. Large Language Model이 Tool 선택을 도운 후, 코드가 이 세션 내에서 실행되고, 합성을 위해 사용자 또는 Agent에게 반환됩니다.

![architecture local](code-interpreter.png)

## 1. 환경 설정

먼저 필요한 라이브러리를 가져오고 Code Interpreter 클라이언트를 초기화합니다.

기본 세션 타임아웃은 900초(15분)입니다. 그러나 데이터에 대한 상세한 분석을 수행할 것이므로 세션 타임아웃 기간을 1200초(20분)로 약간 늘려서 세션을 시작합니다

In [ ]:
!pip install --upgrade -r requirements.txt

In [ ]:
# Bedrock AgentCore Code Interpreter 클라이언트
from bedrock_agentcore.tools.code_interpreter_client import CodeInterpreter
# Langchain Agent 관련 모듈
from langchain.agents import AgentExecutor, create_tool_calling_agent, initialize_agent, tool
# AWS Bedrock Converse API를 위한 Langchain 래퍼
from langchain_aws import ChatBedrockConverse
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
import json
import pandas as pd
from typing import Dict, Any, List

# us-west-2 리전에서 Code Interpreter 초기화
code_client = CodeInterpreter('us-west-2')
# 세션 타임아웃 1200초(20분)로 세션 시작
code_client.start(session_timeout_seconds=1200)

## 2. 로컬 데이터 파일 읽기

이제 샘플 데이터 파일의 내용을 읽습니다. 이 파일은 Name, Preferred_City, Preferred_Animal, Preferred_Thing의 4개 컬럼과 약 300,000개의 레코드로 구성된 랜덤 데이터로 이루어져 있습니다.

조금 후에 Agent를 사용하여 이 파일을 분석하여 분포와 이상치를 이해할 것입니다

In [ ]:
df_data = pd.read_csv("samples/data.csv")
df_data.head()

In [ ]:
def read_file(file_path: str) -> str:
    """Helper function to read file content with error handling"""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return file.read()
    except FileNotFoundError:
        print(f"Error: The file '{file_path}' was not found.")
        return ""
    except Exception as e:
        print(f"An error occurred: {e}")
        return ""

# 로컬 CSV 파일 내용을 문자열로 읽기
data_file_content = read_file("samples/data.csv")

## 3. 샌드박스 환경을 위한 파일 준비

샌드박스 환경에서 생성하려는 파일을 정의하는 구조를 만듭니다.

In [ ]:
# 샌드박스 환경에 생성할 파일 정의 (경로와 내용)
files_to_create = [
                {
                    "path": "data.csv",
                    "text": data_file_content
                }]

## 4. Tool 호출을 위한 헬퍼 함수 생성

이 헬퍼 함수는 샌드박스 Tool을 호출하고 응답을 처리하는 것을 더 쉽게 만들어줍니다. 활성 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간에 상태를 유지할 수 있습니다.

In [ ]:
def call_tool(tool_name: str, arguments: Dict[str, Any]) -> Dict[str, Any]:
    """Helper function to invoke sandbox tools

    Args:
        tool_name (str): Name of the tool to invoke
        arguments (Dict[str, Any]): Arguments to pass to the tool

    Returns:
        Dict[str, Any]: JSON formatted result
    """
    # Code Interpreter의 invoke 메서드 호출
    response = code_client.invoke(tool_name, arguments)
    # 스트림 응답에서 결과를 JSON으로 반환
    for event in response["stream"]:
        return json.dumps(event["result"])

## 5. Code Sandbox에 데이터 파일 작성

이제 데이터 파일을 샌드박스 환경에 작성하고 성공적으로 생성되었는지 확인합니다.

In [ ]:
# writeFiles tool을 사용하여 샌드박스에 파일 작성
writing_files = call_tool("writeFiles", {"content": files_to_create})
print("Writing files result:")
print(writing_files)

# listFiles tool을 사용하여 파일 생성 확인
listing_files = call_tool("listFiles", {"path": ""})
print("\nFiles in sandbox:")
print(listing_files)

## 6. Langchain 기반 Agent를 사용한 고급 분석 수행

이제 위에서 샌드박스에 업로드한 데이터 파일에 대한 데이터 분석을 수행하도록 Agent를 구성합니다

### 6.1 시스템 프롬프트 정의
AI 어시스턴트의 동작과 기능을 정의합니다. 항상 코드 실행과 데이터 기반 추론을 통해 답변을 검증하도록 어시스턴트에게 지시합니다.

In [ ]:
SYSTEM_PROMPT = """You are a helpful AI assistant that validates all answers through code execution using the tools provided. DO NOT Answer questions without using the tools

VALIDATION PRINCIPLES:
1. When making claims about code, algorithms, or calculations - write code to verify them
2. Use execute_python to test mathematical calculations, algorithms, and logic
3. Create test scripts to validate your understanding before giving answers
4. Always show your work with actual code execution
5. If uncertain, explicitly state limitations and validate what you can

APPROACH:
- If asked about a programming concept, implement it in code to demonstrate
- If asked for calculations, compute them programmatically AND show the code
- If implementing algorithms, include test cases to prove correctness
- Document your validation process for transparency
- The sandbox maintains state between executions, so you can refer to previous results

TOOL AVAILABLE:
- execute_python: Run Python code and see output

RESPONSE FORMAT: The execute_python tool returns a JSON response with:
- sessionId: The sandbox session ID
- id: Request ID
- isError: Boolean indicating if there was an error
- content: Array of content objects with type and text/data
- structuredContent: For code execution, includes stdout, stderr, exitCode, executionTime

For successful code execution, the output will be in content[0].text and also in structuredContent.stdout.
Check isError field to see if there was an error.

Be thorough, accurate, and always validate your answers when possible."""

### 6.2 코드 실행 Tool 정의
다음으로 Agent가 Tool로 사용할 함수를 정의하여 코드 샌드박스에서 코드를 실행합니다. @tool 데코레이터를 사용하여 함수를 Agent의 커스텀 도구로 주석 처리합니다.

활성 Code Interpreter 세션 내에서 지원되는 언어(Python, JavaScript)로 코드를 실행하고, 종속성 구성에 따라 라이브러리에 액세스하고, 시각화를 생성하고, 실행 간에 상태를 유지할 수 있습니다.

In [ ]:
#Define and configure the code interpreter tool
@tool
def execute_python(code: str, description: str = "") -> str:
    """Execute Python code in the sandbox."""

    # 설명이 있으면 코드 앞에 주석으로 추가
    if description:
        code = f"# {description}\n{code}"

    # 생성된 코드 출력
    print(f"\n Generated Code: {code}")


    # executeCode tool을 호출하여 Code Interpreter 세션에서 코드 실행
    response = code_client.invoke("executeCode", {
        "code": code,
        "language": "python",
        "clearContext": False  # 실행 간 상태 유지
    })
    # 스트림 응답에서 결과를 JSON으로 반환
    for event in response["stream"]:
        return json.dumps(event["result"])

### 6.3 Agent 구성
Langchain SDK를 사용하여 Agent를 생성하고 구성합니다. 생성된 코드를 실행하기 위해 위에서 정의한 시스템 프롬프트와 Tool을 제공합니다

#### 6.4 언어 모델 초기화

Claude Haiku 4.5 Model을 사용하기 위해 [Cross-region Inference (CRIS)](https://docs.aws.amazon.com/bedrock/latest/userguide/cross-region-inference.html) 프로필 ID를 지정합니다

In [ ]:
# Cross-region Inference (CRIS) 프로필을 사용하여 Claude Haiku 4.5 모델 초기화
llm = ChatBedrockConverse(model_id="global.anthropic.claude-haiku-4-5-20251001-v1:0",region_name="us-west-2")

#### 6.5 프롬프트 템플릿 정의

In [ ]:
# 시스템 프롬프트, 사용자 입력, agent_scratchpad를 포함한 프롬프트 템플릿 정의
prompt = ChatPromptTemplate.from_messages([
    ("system", SYSTEM_PROMPT),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),  # Agent의 중간 추론 과정 저장
])

#### 6.6 커스텀 도구 목록 생성

In [ ]:
tools = [execute_python]

### 6.7 Agent executor 생성

In [ ]:
# Tool calling을 지원하는 Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)
# Agent를 실행할 executor 생성 (verbose=True로 실행 과정 출력)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

## 7. Agent 호출 및 응답 처리
쿼리로 Agent를 호출하고 Agent의 응답을 처리합니다


참고: 비동기 실행은 비동기 환경에서 실행해야 합니다

## 7.1 탐색적 데이터 분석(EDA) 수행 쿼리

코드 샌드박스 환경의 데이터 파일에 대해 탐색적 데이터 분석을 수행하도록 Agent에게 지시하는 쿼리로 시작하겠습니다

In [ ]:
query = "Load the file 'data.csv' and perform exploratory data analysis(EDA) on it. Tell me about distributions and outlier values."

# Agent 실행 및 응답 받기
response=agent_executor.invoke({"input": query})
print("\n*********Final Results*********")
# 응답의 첫 번째 output에서 텍스트 추출
print(response['output'][0]['text'])

## 7.2 정보 추출 쿼리

이제 코드 샌드박스 환경의 데이터 파일에서 특정 정보를 추출하도록 Agent에게 지시하겠습니다

In [ ]:
query = "Within the file 'data.csv', how many individuals with the first name 'Kimberly' have 'Crocodile' as their favourite animal?"

# Agent 실행 및 응답 받기
response=agent_executor.invoke({"input": query})
print("\n*********Final Results*********")
# 응답의 첫 번째 output에서 텍스트 추출
print(response['output'][0]['text'])

## 8. 정리

마지막으로 Code Interpreter 세션을 중지하여 정리합니다. 세션 사용이 끝나면 리소스를 해제하고 불필요한 요금을 피하기 위해 세션을 중지해야 합니다.

In [ ]:
# Code Interpreter 세션 종료 (리소스 해제)
code_client.stop()
print("Code Interpreter session stopped successfully!")